In [ ]:
import pandas as pd
import calendar as cal
import datetime
import re
from glob import glob

## 0. 데이터 가져오기
- 1) 당진시 날씨 예보 데이터
- 2) 당진시 날씨 관측 데이터
- 3) 동서발전 발전소별 발전량 데이터
- 4) sample
- 5) 울산광역시 날씨 예보 데이터
- 6) 울산광역시 날씨 관측 데이터

In [ ]:
path = "./data/"
files = sorted(glob(path+"*.csv"))
files

In [ ]:
# 1. 당진 발전소 동네 날씨 정보
dj_fcst = pd.read_csv(files[0])
dj_fcst.head()

# Forecast time: 예보 발표 시점 2018-03-01 11:00:00
# forcast: 예보 시간 11시에 발표해서 forcast 만큼 지난 시간 +4면 15:00:00
# Temperature: 온도 (도씨)
# Humidity: 습도 (%)
# WindSpeed: 풍속 (m/s)
# WindDirection: 풍향
# Cloud: 하늘 상태
# 1-맑음 / 2-구름보통 / 3-구름많음 / 4-흐림


In [ ]:
# 2. 당진 발전소 인근 기상 관측 정보
dj_obs = pd.read_csv(files[1])
dj_obs.head()

# 지점: 지점 코드
# 지점명: 관측소 지점 (서산)
# 일시: 관측 시간 Y-M-D H:M
# 기온 , 풍속, 풍향, 습도
# 전운량: 낮을수록 구름이 적다.

In [ ]:
# 3. 발전소별 발전량
energy = pd.read_csv(files[2])
energy[8:18]

# time: 1시간 단위 계량된 시간 1:00:00 이면 0:00:00 ~ 1:00:00
# dangjin_floating: 당진수상태양광 발전량(KW)
# dangjin_warehouse: 당진자재창고태양광 발전량(KW)
# dangjin: 당진태양광 발전량(KW)
# ulsan: 울산태양광 발전량(KW)


In [ ]:
# 4. 예측한 발전량 제출 양식
sample_sub = pd.read_csv(files[3])
sample_sub.head(2)

# time: 지난 한시간동안 발전량 예측
# dangjin_floating: 당진수상태양광 예측 발전량 (KW)
# dangjin_warehouse: 당진자재창고태양광 예측 발전량 (KW)
# ulsan: 울산태양광 예측 발전량

In [ ]:
# 5. 발전소 정보
site = pd.read_csv(files[4])
site

# Id: 발전소 식별자
# Capacity: 발전소 발전용량(MW)
# Address: 주소
# InstallationAngle: 설치각
# IncidentAngle: 입사각
# Latitude: 위도
# Longitude: 경도

In [ ]:
# 6. 울산 발전소 동네 날씨 정보
us_fcst = pd.read_csv(files[5])
us_fcst.head()

# Forecast time: 예보 발표 시점 2018-03-01 11:00:00
# forcast: 예보 시간 11시에 발표해서 forcast 만큼 지난 시간 +4면 15:00:00
# Temperature: 온도 (도씨)
# Humidity: 습도 (%)
# WindSpeed: 풍속 (m/s)
# WindDirection: 풍향
# Cloud: 하늘 상태
# 1-맑음 / 2-구름보통 / 3-구름많음 / 4-흐림


In [ ]:
# 7. 울산 발전소 인근 기상 관측 정보
us_obs = pd.read_csv(files[6])
us_obs.head()

# 지점: 지점 코드
# 지점명: 관측소 지점 (울산)
# 일시: 관측 시간 Y-M-D H:M
# 기온 , 풍속, 풍향, 습도
# 전운량: 낮을수록 구름이 적다.

# 1. 당진 발전소 동네 날씨 정보
# dj_fcst = pd.read_csv(files[0])
# 2. 당진 발전소 인근 기상 관측 정보
# dj_obs = pd.read_csv(files[1])
# 3. 발전소별 발전량
# energy = pd.read_csv(files[2])
# 4. 예측한 발전량 제출 양식
# sample_sub = pd.read_csv(files[3])
# 5. 발전소 정보
# site = pd.read_csv(files[4])
# 6. 울산 발전소 동네 날씨 정보
# us_fcst = pd.read_csv(files[5])
# 7. 울산 발전소 인근 기상 관측 정보
# us_obs = pd.read_csv(files[6])

---

## 1. 데이터 정제 (울산광역시 데이터)
### 1) 날씨 정보 컬럼명 변경

In [ ]:
# 울산광역시 us_fcst
# 날씨 예보 데이터 컬럼명 변경
# 읽기 쉽게 한글로 변경
us_fcst.rename(columns={us_fcst.columns[0]: "예보발표시간",
us_fcst.columns[1]:"시간", # 예측시각
us_fcst.columns[2]:"기온",
us_fcst.columns[3]:"습도",
us_fcst.columns[4]:"풍속",
us_fcst.columns[5]:"풍향",
us_fcst.columns[6]:"날씨코드"}, inplace=True)
us_fcst.head()


In [ ]:
# 울산광역시 us_obs
# 날씨 관측 데이터 컬럼명 변경
us_obs.rename(columns={us_obs.columns[0]: "지점코드",
us_obs.columns[1]:"지점명",
us_obs.columns[2]:"시간", # 관측시간
us_obs.columns[3]:"기온",
us_obs.columns[4]:"풍속",
us_obs.columns[5]:"풍향",
us_obs.columns[6]:"습도",
us_obs.columns[7]:"전운량"}, inplace=True)
us_obs.head()

----
### 2) 예보시간 TimeStamp로 나타내기
- 2018-03-01 11시에 관측하여
- 시간 컬럼이 만약에 4라면
- 2018-03-01 15시 의 timestamp 값을 가지도록

In [ ]:
# 울산 광역시 us_fcst
# 날씨 예보 데이터 시간 컬럼 -> TimeStamp
us_fcst.head()

In [ ]:
# 예보발표시간 날짜에 해당하는 월의 마지막 일 수 구하기
#  2018 3월 ~ 2021 3월
months = [i for i in range(1, 13)]
lastDays = {2018: [], 2019: [], 2020: [], 2021: []}

days = []
for year in lastDays:
    for month in months:
        lastDays[year].append(cal.monthrange(year, month)[1])

print(lastDays)

In [ ]:
# 울산광역시 us_fcst
# 반복문을 사용하여 예측 시간 열을 timestamp로 변경
for i in range(len(us_fcst)):
    announceTime = us_fcst["예보발표시간"][i] # 예보를 발표한 시간
    predictTime = us_fcst["시간"][i] # 예보 시각
    annTS = pd.Timestamp(announceTime) # 예보발표시간 to TimeStamp

    year = annTS.year # 연도
    month = annTS.month # 월
    day = annTS.day # 일
    hour = annTS.hour # 시

    # 예보 시각의 hour 구하기
    # 예보발표시간 + 시간 열
    preHour = int(hour + predictTime)
    plusDay = preHour // 24 # 더해지는 일수 (24시가 지나면 + 1)
    hour = preHour % 24 # 나머지 시간

    # 만약 24시가 지났다면
    if(plusDay > 0):
        day += plusDay # 해당 일 + 더해지는 일 수
        # 만약 month가 1, 3, 5, 7, 8, 10, 12 라면: 31일
        if(month == 1 or month == 3 or month == 5 or month == 7 or month == 8 or month == 10 or month == 12):
            if(day > 31):
                month += 1
                day %= 31
                if(month> 12):
                    year += 1
                    month = 1
        # 만약 month가 2, 4, 6, 9, 11 이라면: 31일 X
        else:
            lastDay = lastDays[year][month-1] # 월의 마지막 일 구하기
            if(day > lastDay):
                month += 1
                day %= lastDay
                if(month> 12):
                    year += 1
                    month = 1
        preTS = pd.Timestamp(year, month, day, hour) # 예보시각 -> 타임 스탬프
        preTS = datetime.datetime.strftime(preTS, '%Y-%m-%d %H:%M') # 2021-04-19 18:00 이런식으로 양식 변경
        us_fcst['시간'][i] = preTS # 날짜 예보 데이터의 시간 열에 넣어준다.
    else:
        preTS = pd.Timestamp(year, month, day, hour)
        preTS = datetime.datetime.strftime(preTS, '%Y-%m-%d %H:%M')
        us_fcst['시간'][i] = preTS


In [ ]:
# 시간 컬럼 -> TimeStamp 변경 결과
us_fcst.head()

In [ ]:
# 변경된 데이터 파일로 저장하기
# # 파일로 변환
us_fcst.to_csv('ulsan_fcst_convert.csv', mode='w', encoding='utf-8-sig', index=False)

In [ ]:
# 저장된 파일 가져오기
us_fcst_convert = pd.read_csv('./ulsan_fcst_convert.csv')
us_fcst_convert = us_fcst_convert.sort_values(by="시간")
us_fcst_convert = us_fcst_convert.reset_index(drop=True, inplace=False)
us_fcst_convert.head()

------
### 3) 날씨 예보 데이터 & 날씨 관측 데이터 Merge
- 필요없는 컬럼명 drop

In [ ]:
# # 시간 컬럼을 기준으로 merge 한다.
# merge_us = pd.merge(us_fcst_convert, us_obs, on='시간')
# merge_us.head()

In [ ]:
# 울산광역시 us_obs
# 날씨 관측 데이터 필요없는 컬럼 삭제
# 지점코드, 지점명 삭제
us_obs_drop = us_obs.drop(['지점코드', '지점명'], axis=1)
us_obs_drop.head()

In [ ]:
# 울산광역시 us_fcst
# 날씨 관측 데이터 필요없는 컬럼명 삭제
# 예보발표시간
us_fcst_drop = us_fcst_convert.drop(['예보발표시간'], axis=1)
us_fcst_drop.head()

In [ ]:
# 울산광역시 us_fcst, us_obs
# 예보 데이터 , 관측 데이터 시간 컬럼을 기준으로 Merge
# _x : 예보 데이터
# _y : 관측 데이터
merge_us = pd.merge(us_fcst_drop, us_obs_drop, how='left', left_on='시간', right_on='시간')
merge_us.head()

In [ ]:
# merge한 데이터 컬럼명 변경
# 예측 / 관측 컬럼
merge_us = merge_us.rename(columns={merge_us.columns[1]:"예측기온", merge_us.columns[2]:"예측습도",merge_us.columns[3]:"예측풍속",
merge_us.columns[4]:"예측풍향",
merge_us.columns[6]:"관측기온",merge_us.columns[7]:"관측풍속",
merge_us.columns[8]:"관측풍향",merge_us.columns[9]:"관측습도",})
merge_us

In [ ]:
# merge한 데이터 NaN 값 삭제
# 행 기준
merge_us = merge_us.dropna(axis=0)
merge_us

---
### 4) 에너지 데이터 가공
- 울산 데이터

In [ ]:
# 울산광역시 us_fcst, us_obs
# 병합한 예보 / 관측 데이터 파일로 저장
merge_us.to_csv('ulsan_pre_obs.csv', mode='w', encoding='utf-8-sig', index=False)

In [ ]:
# 동서발전 발전소별 발전량 데이터 energy
# 울산 데이터만 뽑기
energy_us = energy.drop(['dangjin_floating', 'dangjin_warehouse', 'dangjin'], axis=1)
energy_us = energy_us.rename(columns={energy_us.columns[0]:'시간',energy_us.columns[1]:'발전량'}, )
energy_us.head()

In [ ]:
# 동서발전 발전소별 발전량 데이터 시간 변경
# 현재로는 H:M:S 양식이며, 00시가 아닌 24시로 표시되어 있음.
# 1) H:M 양식의 타임스탬프 변경
# 2) 24시 -> 00시로 변경
for i in range(len(energy_us)):
    energy_us['시간'][i] = re.sub("24:00:00", "00:00:00", energy_us['시간'][i])
    time = pd.Timestamp(energy_us['시간'][i])
    time = datetime.datetime.strftime(time, '%Y-%m-%d %H:%M')
    energy_us['시간'][i] = time

In [ ]:
# 데이터 변경 확인
energy_us.loc[22:26]

In [ ]:
# 발전량이 0인 데이터 삭제
energy_us = energy_us[energy_us['발전량'] != 0]
energy_us.head()

---
### 5) energy 데이터와 병합

In [ ]:
# 울산광역시 예측 - 관측 데이터 us_pre_obs
# 파일로 저장한 데이터 가져오기
us_pre_obs = pd.read_csv('./ulsan_pre_obs.csv')
us_pre_obs.head()

In [ ]:
# 울산 광역시 발전량 데이터 energy_us
# 울산 광역시 예측 - 관측 데이터 us_pre_obs
# 두 데이터를 시간을 기준으로 merge 한다.
wheather_energy_df = pd.merge(pre_obs, energy_us, how='left', left_on='시간', right_on='시간')
wheather_energy_df

In [ ]:
# NaN 값 삭제
wheather_energy_df = wheather_energy_df.dropna(axis=0)
wheather_energy_df

In [ ]:
# 울산 광역시 예측 - 관측 - 발전량 데이터
# 파일로 저장
wheather_energy_df.to_csv('ulsan_whether_energy.csv', mode='w', encoding='utf-8-sig', index=False)
wheather_energy = pd.read_csv('ulsan_whether_energy.csv')
wheather_energy

In [ ]:
# wheather_energy.describe()
wheather_energy['시간'][0][11:]

In [ ]:
# 울산 광역시 예측 - 관측 - 발전량 데이터 wheather_energy
# 시간대 데이터 추가하기
wheather_energy['시간대'] = -1
for i in range(len(wheather_energy)):
    time = wheather_energy['시간'][i][11:]
    if(time == ('06:00') or time == ('07:00') or time == ('08:00')):
        wheather_energy['시간대'][i] = '6 - 8시'
    elif (time == ('09:00') or time == ('10:00') or time == ('11:00')):
        wheather_energy['시간대'][i] = '9 - 11시'
    elif (time == ('12:00') or time == ('13:00') or time == ('14:00')):
        wheather_energy['시간대'][i] = '12 - 14시'
    elif (time == ('15:00') or time == ('16:00') or time == ('17:00')):
        wheather_energy['시간대'][i] = '15 - 17시'
    elif (time == ('18:00') or time == ('19:00') or time == ('20:00')):
        wheather_energy['시간대'][i] = '18 - 20시'
    else:
        wheather_energy['시간대'][i] = -1

In [ ]:
# 시간대 열 추가 확인
wheather_energy

In [ ]:
# 시간대를 기준으로 그룹핑
# 발전량의 평균과 총 합을 구한다.
res = wheather_energy.groupby(['시간대']).apply(lambda x: pd.Series({'평균': x['발전량'].mean(), '총발전량': sum(x['발전량'])}))
res = res.sort_values(by='총발전량', ascending=False)
res

## 3. 데이터 시각화
- 관측 기온/습도/풍속/풍향/습도 에 따른 발전량
---
### 0) 모듈 불러오기

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline


In [ ]:
# from matplotlib import font_manager, rc
# font_path = "C:/Windows/Fonts/malgun.ttf"
# font_name= font_manager.FontProperties(fname=font_path).get_name()
# rc('font', family=font_name)
plt.rc('font', family='Malgun Gothic')

In [ ]:
# 시간대에 따른 총 발전량 그래프
plt.figure()
res['총발전량'].plot(kind='barh', grid=True, figsize=(5, 5))
plt.show()

In [ ]:
wheather_energy_test = wheather_energy.drop(['예측기온', '예측습도', '예측풍속', '예측풍향', '날씨코드'], axis=1)
wheather_energy_test

In [ ]:
plt.rc('font', family='Malgun Gothic')
sns.pairplot(wheather_energy_test, kind="hist", hue="시간대")
plt.show()